# JSON library 
1. is a serialization standard
2. uses text encoding
3. language independent
4. consists of
    - root is an object 
    - objects: unordered key:value pairs delimited by {}
    - array: ordered list of elements seperated by , and delimited by []
    - values(numbers, strings delimited by double quotes, boolean, null, objects, arrays)
5. whitespaces do not matter. 

In [5]:
s = '''
{
  "array": [
    1,
    2,
    3
  ],
  "boolean": true,
  "color": "gold",
  "null": null,
  "number": 123.45,
  "object": {
    "a": "b",
    "c": "d"
  },
  "string": "Hello World"
}
'''

In [6]:
import json

In [8]:
j_obj = json.loads(s)

j_obj

{'array': [1, 2, 3],
 'boolean': True,
 'color': 'gold',
 'null': None,
 'number': 123.45,
 'object': {'a': 'b', 'c': 'd'},
 'string': 'Hello World'}

In [11]:
print(json.dumps(j_obj, indent=2))

{
  "array": [
    1,
    2,
    3
  ],
  "boolean": true,
  "color": "gold",
  "null": null,
  "number": 123.45,
  "object": {
    "a": "b",
    "c": "d"
  },
  "string": "Hello World"
}


# Python has a lot more objects like datetime objects which cant be serialized cleanly

In [12]:
import datetime

In [64]:
j_obj = {
    "name": "Isaac Newton",
    "dob": datetime.datetime(1643, 1, 4)
}

In [65]:
j_obj['dob'].date().year #so valid python object and we can work on it in python

1643

In [66]:
json.dumps(j_obj)

TypeError: Object of type datetime is not JSON serializable

# we can create a function which json library will send any obj it cant serialize to and our function should return a serialized version back

In [67]:
def my_encoder(obj):
    if isinstance(obj, datetime.datetime):
        return obj.isoformat()
    raise TypeError('Cannot be serialized by my_encoder')

In [68]:
json.dumps(j_obj, default=my_encoder)

'{"name": "Isaac Newton", "dob": "1643-01-04T00:00:00"}'

# lets add a numpy array to the mix

In [69]:
import numpy as np

In [70]:
j_obj = { "type": "job",
    "created_at": datetime.datetime(1970, 1, 1),
    "status": "🆗",
    "payload": np.array([[1, 2], [3, 4]]),
}

In [71]:
json.dumps(j_obj, default=my_encoder)

TypeError: Cannot be serialized by my_encoder

# we can either keep on refining our encoder or use a modern library

In [76]:
import orjson

In [77]:
j_obj = { "type": "job",
    "created_at": datetime.datetime(1970, 1, 1),
    "status": "🆗",
    "payload": np.array([[1, 2], [3, 4]]),
}

In [78]:
orjson.dumps(j_obj, option=orjson.OPT_NAIVE_UTC | orjson.OPT_SERIALIZE_NUMPY)

b'{"type":"job","created_at":"1970-01-01T00:00:00+00:00","status":"\xf0\x9f\x86\x97","payload":[[1,2],[3,4]]}'

# REST API

In [81]:
import os
from dotenv import load_dotenv 

load_dotenv()

FINHUBAPI = os.getenv('FINHUBAPI')
X_Finnhub_Secret = os.getenv('X-Finnhub-Secret')

### finnhub.io : All GET request require a token parameter token=apiKey in the URL or a header X-Finnhub-Token : apiKey

In [95]:
import requests

In [96]:
payload = {'symbol': 'AAPL', 'token': FINHUBAPI}
r = requests.get('https://finnhub.io/api/v1/quote', params=payload)

In [97]:
# to check if request has been correctly formed
r.url

'https://finnhub.io/api/v1/quote?symbol=AAPL&token=d3r8f51r01qopgh7f7f0d3r8f51r01qopgh7f7fg'

In [99]:
r.status_code

200

In [100]:
r.text

'{"c":263.47,"d":11.18,"dp":4.4314,"h":264.375,"l":253.4,"o":253.4,"pc":252.29,"t":1760988480}'

In [98]:
# built in json decoder
r.json()

{'c': 263.47,
 'd': 11.18,
 'dp': 4.4314,
 'h': 264.375,
 'l': 253.4,
 'o': 253.4,
 'pc': 252.29,
 't': 1760988480}

In [103]:
r.headers

{'Date': 'Mon, 20 Oct 2025 19:28:41 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Server': 'cloudflare', 'Nel': '{"report_to":"cf-nel","success_fraction":0.0,"max_age":604800}', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Headers': 'Origin', 'Access-Control-Allow-Methods': 'POST, GET, OPTIONS, PUT, DELETE', 'Access-Control-Allow-Origin': '*', 'Access-Control-Expose-Headers': 'Content-Length', 'X-Ratelimit-Limit': '60', 'X-Ratelimit-Remaining': '59', 'X-Ratelimit-Reset': '1760988581', 'cf-cache-status': 'DYNAMIC', 'Report-To': '{"group":"cf-nel","max_age":604800,"endpoints":[{"url":"https://a.nel.cloudflare.com/report/v4?s=7WisfnZBJptfktAoLqmB%2Fs7FbKsS4Nj4kAh9bCCcy%2BD4NK56NyZOh6RCQZNZRTyMPhh7mdPiikk8Wi5CacNuud1YNeKEbn%2FPs9I%3D"}]}', 'Content-Encoding': 'gzip', 'CF-RAY': '991aeb70c9462561-LHR', 'alt-svc': 'h3=":443"; ma=86400'}

In [106]:
r.cookies

<RequestsCookieJar[]>

In [112]:
url = 'https://finnhub.io/api/v1/quote'
payload = {'symbol': 'AAPL'}
headers = {'X-Finnhub-Secret': FINHUBAPI}

r = requests.get(url, params=payload, headers=headers)
r.json()

{'error': 'Please use an API key.'}

In [113]:
r.status_code

401

In [114]:
response = requests.get(
    url='https://www.google.com/search',
    params={'q': 'python http requests', 'num': 5},
)

response.status_code

200

In [115]:
response.reason

'OK'

In [109]:
response.text

'<!DOCTYPE html><html lang="en" dir="ltr"><head><style nonce="KBxRP6CtEGnjkIXXnnzN-g">\na, a:link, a:visited, a:active, a:hover {\n  color: #1a73e8;\n  text-decoration: none;\n}\nbody {\n  font-family: Roboto, Helvetica, Arial, sans-serif;\n  text-align: center;\n  -ms-text-size-adjust: 100%;\n  -moz-text-size-adjust: 100%;\n  -webkit-text-size-adjust: 100%;\n}\n.youtubeContainerUIModernization,\n.boxUIModernization {\n  box-sizing: border-box;\n  margin-left: auto;\n  margin-right: auto;\n  max-width: 800px;\n}\n.signInContainerUIModernization {\n    display: flex;\n    justify-content: flex-end;\n}\nh1 {\n  color: #2c2c2c;\n  font-size: 24px;\n  hyphens: auto;\n  margin: 24px 0;\n}\n.openInNewIcon,\n.openInNewIconYouTube {\n  height: 12px;\n  width: 12px;\n  margin-bottom: -2px;\n  margin-left: 2px;\n\n  \n}\n.openInNewIcon {\n  fill: #0b57d0; // gm3-primary-blue\n}\n.openInNewIconYouTube {\n  fill: #065fd4; // yt-primary-dark-blue\n}\n.icaCallout {\n  background-color: #f8f9fa;\n  p

In [111]:
for key, value in response.headers.items():
    print(f'{key}: {value}')

Content-Type: text/html; charset=utf-8
Cache-Control: no-cache, no-store, max-age=0, must-revalidate
Pragma: no-cache
Expires: Mon, 01 Jan 1990 00:00:00 GMT
Date: Mon, 20 Oct 2025 19:31:46 GMT
P3P: CP="This is not a P3P policy! See g.co/p3phelp for more info."
Cross-Origin-Resource-Policy: same-site
Content-Security-Policy: require-trusted-types-for 'script';report-uri /_/ConsentHttp/cspreport, script-src 'nonce-hVPjnXEvN47hD4280lrO1Q' 'unsafe-inline';object-src 'none';base-uri 'self';report-uri /_/ConsentHttp/cspreport;worker-src 'self'
Permissions-Policy: ch-ua-arch=*, ch-ua-bitness=*, ch-ua-full-version=*, ch-ua-full-version-list=*, ch-ua-model=*, ch-ua-wow64=*, ch-ua-form-factors=*, ch-ua-platform=*, ch-ua-platform-version=*
Accept-CH: Sec-CH-UA-Arch, Sec-CH-UA-Bitness, Sec-CH-UA-Full-Version, Sec-CH-UA-Full-Version-List, Sec-CH-UA-Model, Sec-CH-UA-WoW64, Sec-CH-UA-Form-Factors, Sec-CH-UA-Platform, Sec-CH-UA-Platform-Version
Cross-Origin-Opener-Policy: unsafe-none
reporting-endpoin